In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

today = pd.Timestamp.today()

In [2]:
WO_BOM = pd.read_pickle('data/temp01/WO_BOM.pkl')
EXCLUDED_PARTS = pd.read_pickle('data/temp01/EXCLUDED_PARTS.pkl')
#ALTERNATES = pd.read_pickle('data/temp02/ALTERNATES.pkl')
WO_HEADER = pd.read_pickle('data/temp02/WO_HEADER.pkl')
PMA_ACCEPT = pd.read_pickle('data/temp02/PMA_ACCEPTANCE.pkl')
BEST_PNM = pd.read_pickle('data/temp02/BEST_PNM.pkl')
PARTS_MASTER = pd.read_pickle('data/temp03/PARTS_MASTER.pkl')

WO_BOM['COMMENT'] = np.NaN

#check against Master
WO_BOM = WO_BOM.merge(PARTS_MASTER.drop_duplicates(subset=['PN'],keep='first')[['PN','CODE']],
                      left_on='PN',right_on='PN',how='left')
WO_BOM['COMMENT'] = np.where(WO_BOM['CODE'].isnull(), 'TO WARN: PN not in master', WO_BOM['COMMENT'])

#check against WO MASTER
WO_HEADER.drop(columns=['DESCRIPTION'],inplace=True)
WO_HEADER.rename(columns={'PN':'LRU_PN','COMMENT':'COMMENT_WOH','DATA_SOURCE':'DATA_SOURCE_WOH','PNM':'LRU_PNM'},inplace = True)
WO_BOM.rename(columns={'DATA_SOURCE':'DATA_SOURCE_WOB','COMMENT':'COMMENT_WOB'},inplace=True)
WO_BOM.drop(columns=['STATUS','DESCRIPTION','GEO_CODE','DEPT_NAME'],inplace=True)
WO_BOM = WO_BOM.merge(WO_HEADER,left_on='WONO',right_on='WONO', how='left')
WO_BOM['COMMENT_WOB'] = np.where(WO_BOM['STATUS'].isnull(), 'TO REMOVE: WO NOT IN WO_HEADER', WO_BOM['COMMENT_WOB'])


#bring top PNM for parts and rename columns
WO_BOM = WO_BOM.merge(BEST_PNM,left_on='PNM',right_on='PNM',how='left')
WO_BOM.drop(columns=['PNM'],inplace = True)
WO_BOM.rename(columns={'BEST_PNM':'PNM'},inplace = True)

#bring top PNM for LRU and rename columns
WO_BOM = WO_BOM.merge(BEST_PNM,left_on='LRU_PNM',right_on='PNM',how='left')
WO_BOM.drop(columns=['PNM_y','LRU_PNM'],inplace = True)
WO_BOM.rename(columns={'BEST_PNM':'LRU_PNM','PNM_x':'PNM'},inplace = True)


In [3]:
#Change PMA to OEM if not in approved list
WO_BOM['CORRECTED_PNM'] = WO_BOM['PNM']
#si le PN est un PMA, voir si il est dans la liste = comparer CUSTNO, PNM, LRU_PNM
PMA_ACCEPT.rename(columns={'custno':'COMPANY_CODE'},inplace=True)
PMA_ACCEPT['VALID_PMA'] = True
PMA_ACCEPT.drop(columns=['PN','LRU_PN'],inplace = True)
WO_BOM = WO_BOM.merge(PMA_ACCEPT,left_on=['PNM','LRU_PNM','COMPANY_CODE'],
                      right_on=['PNM','LRU_PNM','COMPANY_CODE'],how='left')

WO_BOM['VALID_PMA'] = np.where((WO_BOM['VALID_PMA'].isnull()) & (WO_BOM['CODE'] == 'PM'),
                              False, WO_BOM['VALID_PMA'])

#bring top alts
WO_BOM = WO_BOM.merge(PARTS_MASTER[['PNM','ALT_PNM','ALT_PNM_OE','ALT_PNM_PM']],left_on = 'PNM',right_on='PNM',how ='left')

#Change PMA to top OEM alt if not in approved list 
WO_BOM['CORRECTED_PNM'] = np.where(WO_BOM['VALID_PMA'] == False, WO_BOM['ALT_PNM_PM'],WO_BOM['CORRECTED_PNM'])
WO_BOM['COMMENT_WOB'] = np.where(WO_BOM['VALID_PMA'] == False,
                                'PMA CONSUMPTION NOT APPROVED', WO_BOM['COMMENT_WOB'])


#Change OEM to top PMA alt if not in approved list 
WO_BOM['CORRECTED_PNM'] = np.where((WO_BOM['VALID_PMA'] == True) & (WO_BOM['CODE'] == 'OE'),
                                   WO_BOM['ALT_PNM_PM'],WO_BOM['CORRECTED_PNM'])
WO_BOM['COMMENT_WOB'] = np.where((WO_BOM['VALID_PMA'] == True) & (WO_BOM['CODE'] == 'OE'),
                                'OEM CHANGED TO APROVED PMA', WO_BOM['COMMENT_WOB'])


,WONO,PN,TASK,ACTIVITY,QTY_REQ,QTY_ISSUED,QTY_RESERVED,COND,COST,ADD_DATE,...,VIP_STATUS,VIP_STATUS_DATE,Termination date,PNM,LRU_PNM,CORRECTED_PNM,VALID_PMA,ALT_PNM,ALT_PNM_OE,ALT_PNM_PM
358,156310,119725-1-AT,BOM,Turn-In,0.0,0.0,0.0,NE,0.0,2017-04-25,...,ARO,2017-03-03,NaT,13594,13435,NaN,True,13592,13592,NaN
360,155728,119725-1-AT,BOM,Consumable,0.0,0.0,0.0,SV,0.0,2017-03-29,...,ARO,2017-03-03,NaT,13594,13435,NaN,True,13592,13592,NaN
361,155132,119725-1-AT,BOM,Work Order,0.0,0.0,0.0,AR,0.0,2017-03-01,...,ARO,2017-03-03,NaT,13594,13423,NaN,True,13592,13592,NaN
362,153960,119725-1-AT,BOM,Consumable,1.0,0.0,0.0,NE,NaN,2017-01-11,...,ARO,2017-03-03,NaT,13594,13435,NaN,True,13592,13592,NaN
384,150532,119725-1-AT,BOM,Consumable,1.0,0.0,0.0,NE,NaN,2016-08-11,...,ARO,2017-03-03,NaT,13594,13435,NaN,True,13592,13592,NaN
1227,152248,273T6302-2,BOM,Consumable,1.0,0.0,0.0,NE,NaN,2016-10-27,...,NaN,NaT,NaT,NaN,48288,NaN,NaN,NaN,NaN,NaN
1959,152963,3234804-1WD,BOM,Consumable,1.0,0.0,0.0,NE,NaN,2016-11-29,...,NaN,NaT,NaT,63156,14947,NaN,False,63155,63155,NaN
1960,150355,3234804-1WD,BOM,Consumable,1.0,0.0,0.0,NE,NaN,2016-08-04,...,ARO,2017-03-03,NaT,63156,14946,NaN,False,63155,63155,NaN
2175,154161,3238329-27STAR,BOM,Consumable,1.0,0.0,0.0,NE,NaN,2017-01-19,...,ARO,2017-03-03,NaT,63388,64021,NaN,False,63387,63387,NaN
2442,155947,119725-1-AT,BOM,Consumable,1.0,1.0,0.0,SR,0.0,2017-05-09,...,ARO,2017-03-03,NaT,13594,13435,NaN,True,13592,13592,NaN


In [18]:
#Remove Excluded items from EXCLUDED PARTS list
